In [1]:
%load_ext autoreload
%autoreload 2

# ML libraries 
from sklearn.cluster import KMeans
from sklearn.neighbors.kde import KernelDensity


# Computation & Signal Processing
from scipy import signal
import numpy as np
import pandas as pd
import pylab as pl
import pickle

#biosppy package for ecg signal analysis
from biosppy import storage
from biosppy.signals import ecg

# Plotting
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
import plotly.io as pio
plotly.offline.init_notebook_mode(connected=True)
%matplotlib inline

from utils import * #import data import, clean up and sampling functions. 
# import time
import time
import os

from ECG_feature_extractor_1000 import *
# fs: sampling frequency
# df: reading a csv file and store it in panads dataframe (http://pandas.pydata.org/pandas-docs/stable/)

start = time.time()
fs_acc = 148.148 #actual 148.15 
fs_emg = 1925.9258 #from 1/df['X[s]'][1] 
fs_ecg = 1925.9258

gdrive = True
data_dir = 'csv'

# Specifiying directory
#file = data_dir*gdrive + '/OA_2019_302_P1_Rep_1.3.csv'
#file2 = data_dir*gdrive + '/OA_2019_302_NW2_Rep_1.5.csv'
#file3 = data_dir*gdrive + '/OA_2019_302_P1_Rep_1.3.csv'
#file4 = data_dir*gdrive + '/OA_2019_302_P2_Rep_1.4.csv'






/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.neighbors.kde module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# loading and parsing data
df = load_data('/302_NW1_Rep_1.2.csv', data_dir )
df2 = load_data('/302_NW2_Rep_1.5.csv', data_dir)
df3 = load_data('/302_P1_Rep_1.3.csv', data_dir)
df4 = load_data('/302_P2_Rep_1.4.csv', data_dir)

In [ ]:
emgnw1 = delsys_cleanup(df, column='R RECTUS FEMORIS: EMG 1')

In [3]:
newdf = delsys_cleanup(df, column='all') 

In [4]:
newdf

,X[s],R RECTUS FEMORIS: EMG 1,X[s].1,R RECTUS FEMORIS: Acc 1.X,X[s].2,R RECTUS FEMORIS: Acc 1.Y,X[s].3,R RECTUS FEMORIS: Acc 1.Z,X[s].4,L RECTUS FEMORIS: EMG 2,...,X[s].56,Trigno Trigger sensor 15: Trig 15,X[s].57,L PECTORALIS MAJOR: EKG 16,X[s].58,L PECTORALIS MAJOR: Acc 16.X,X[s].59,L PECTORALIS MAJOR: Acc 16.Y,X[s].60,L PECTORALIS MAJOR: Acc 16.Z
0,0.00052,0.00000,0.00675,0.00000,0.00675,0.00000,0.00675,0.00000,0.00052,0.00000,...,0.000519,0.072555,0.000519,0.000000,0.00675,0.000000,0.00675,0.000000,0.00675,0.000000
1,0.00104,0.00000,0.01350,0.00000,0.01350,0.00000,0.01350,0.00000,0.00104,0.00000,...,0.001038,0.072555,0.001038,0.000000,0.01350,0.000000,0.01350,0.000000,0.01350,0.000000
2,0.00156,0.00000,0.02025,0.00000,0.02025,0.00000,0.02025,0.00000,0.00156,0.00000,...,0.001558,0.072555,0.001558,0.000000,0.02025,0.000000,0.02025,0.000000,0.02025,0.000000
3,0.00208,0.00000,0.02700,-0.89833,0.02700,0.06043,0.02700,-0.14905,0.00208,0.00000,...,0.002077,0.072555,0.002077,0.000000,0.02700,0.060426,0.02700,-0.636484,0.02700,0.306157
4,0.00260,0.00000,0.03375,-0.89833,0.03375,0.05237,0.03375,-0.15308,0.00260,0.00000,...,0.002596,0.072555,0.002596,0.000000,0.03375,0.060426,0.03375,-0.636484,0.03375,0.314214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577791,300.00740,0.00002,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,300.00740,-0.00001,...,300.007400,0.072555,300.007400,0.000156,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000
577792,300.00790,0.00002,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,300.00790,-0.00001,...,300.007900,0.072555,300.007900,0.000164,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000
577793,300.00840,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,300.00840,-0.00001,...,300.008400,0.072555,300.008400,0.000171,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000
577794,300.00890,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,300.00890,-0.00001,...,300.008900,0.072555,300.008900,0.000176,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000


In [ ]:
emg_ecg_columns = ['X[s]', 'R RECTUS FEMORIS: EMG 1', 'L RECTUS FEMORIS: EMG 2', 'Trigno sensor 3: EMG 3', 
                 'L BICEPS FEMORIS: EMG 4', 'Trigno Mini sensor 5: EMG 5', 'R BICEPS FEMORIS: EMG 6',
                 'R TIBIALIS ANTERIOR: EMG 7','L TIBIALIS ANTERIOR: EMG 8', 'L GASTROCNEMIUS LATERAL HEAD: EMG 11',
                 'L GASTROCNEMIUS MEDIAL HEAD: EMG 12', 'R GASTROCNEMIUS LATERAL HEAD: EMG 13',
                 'R GASTROCNEMIUS MEDIAL HEAD: EMG 14', 'L PECTORALIS MAJOR: EKG 16']
acc_columns = ['X[s].1', 'L GASTROCNEMIUS LATERAL HEAD: Acc 11.X', 'L GASTROCNEMIUS LATERAL HEAD: Acc 11.Y',
          'L GASTROCNEMIUS LATERAL HEAD: Acc 11.Z', 'L GASTROCNEMIUS LATERAL HEAD: Acc 11.X',
          'L GASTROCNEMIUS LATERAL HEAD: Acc 11.Y', 'L GASTROCNEMIUS LATERAL HEAD: Acc 11.Z', 
          'L GASTROCNEMIUS MEDIAL HEAD: Acc 12.X', 'L GASTROCNEMIUS MEDIAL HEAD: Acc 12.Y', 
          'L GASTROCNEMIUS MEDIAL HEAD: Acc 12.Z', 'R GASTROCNEMIUS LATERAL HEAD: Acc 13.X', 
          'R GASTROCNEMIUS LATERAL HEAD: Acc 13.Y', 'R GASTROCNEMIUS LATERAL HEAD: Acc 13.Z',
          'R GASTROCNEMIUS MEDIAL HEAD: Acc 14.X', 'R GASTROCNEMIUS MEDIAL HEAD: Acc 14.Y', 
          'R GASTROCNEMIUS MEDIAL HEAD: Acc 14.Z', 'Trigno Trigger sensor 15: Trig 15',
          'L PECTORALIS MAJOR: Acc 16.X', 'L PECTORALIS MAJOR: Acc 16.Y', 'L PECTORALIS MAJOR: Acc 16.Z']

In [13]:
emg_and_ecg = {'nw1':df[emg_ecg_columns], 'nw2':df2[emg_ecg_columns], 'p1':df3[emg_ecg_columns], 'p2':df4[emg_ecg_columns]}

In [14]:
acc = {'nw1':df[acc_columns], 'nw2':df2[acc_columns], 'p1':df3[acc_columns], 'p2':df4[acc_columns]}

In [17]:
time_sync_folders_names = os.listdir('Sync_txt_file'+'/OA_302_B_WMP')
time_sync_folders_names

['N1', 'N2', 'P1', 'P2']

In [18]:
data_dir_treadmill = 'Treadmill_data'
file3_rawdata = '/OA_302_P1_RAWDATA.csv'
treadmill_data = load_data(file3_rawdata, data_dir_treadmill)
treadmill_data = treadmill_data.rename(columns={'Time': 'time'})

In [21]:
data_dir_fNIRS = 'fNIRS_Walking data'
file3_fNIRS = '/OA_FNIRS_2019_WALK_302_oxydata.txt'
dffNIRS = pd.read_csv(data_dir_fNIRS+file3_fNIRS,sep='\t')

In [24]:
dffNIRS = dffNIRS.rename(columns={'1':'subject_ID', '2':'cohort', '3':'block','4':'trial_i','5':'HbO1','6':'HbR1','7':'HbO2','7':'HbR2','8':'HbO3','9':'HbR3','10':'HbO4','11':'HbR4','12':'HbO5','13':'HbR5','14':'HbO6','15':'HbR6','16':'HbO7','17':'HbR7','18':'HbO8','19':'HbR8','20':'HbO9','21':'HbR9','23':'HbO10','24':'HbR10','25':'HbO11','26':'HbR11','27':'HbO12','28':'HbR12','29':'HbO13','30':'HbR13','31':'HbO14','32':'HbR14','33':'HbO15','34':'HbR15','35':'HbO16','36':'HbR16'})
dffNIRS['time'] = [i*0.5 for i in range(len(dffNIRS))]
dffNIRS = dffNIRS[['time','subject_ID', 'cohort', 'block', 'trial_i', 'HbO1', 'HbR1', 'HbR2',
       'HbO3', 'HbR3', 'HbO4', 'HbR4', 'HbO5', 'HbR5', 'HbO6', 'HbR6', 'HbO7',
       'HbR7', 'HbO8', 'HbR8', 'HbO9', 'HbR9', '22', 'HbO10', 'HbR10', 'HbO11',
       'HbR11', 'HbO12', 'HbR12', 'HbO13', 'HbR13', 'HbO14', 'HbR14', 'HbO15',
       'HbR15', 'HbO16', 'HbR16']]

In [25]:
fnirs_trial1 = dffNIRS[dffNIRS['trial_i']==1]
fnirs_trial2 = dffNIRS[dffNIRS['trial_i']==2]
fnirs_trial3 = dffNIRS[dffNIRS['trial_i']==3]
fnirs_trial4 = dffNIRS[dffNIRS['trial_i']==4]
fnirs_trial5 = dffNIRS[dffNIRS['trial_i']==5]
fnirs_trial6 = dffNIRS[dffNIRS['trial_i']==6]
fnirs_trial7 = dffNIRS[dffNIRS['trial_i']==7]
fnirs_trial8 = dffNIRS[dffNIRS['trial_i']==8]
fnirs_trial9 = dffNIRS[dffNIRS['trial_i']==9]
fnirs_trial10 = dffNIRS[dffNIRS['trial_i']==10]

In [26]:
fNIRS = {'nw1':fnirs_trial1, 'nw2':fnirs_trial2, 'p1':fnirs_trial3, 'p2':fnirs_trial4, 'pl':fnirs_trial5, 'ph':fnirs_trial6, 'cl':fnirs_trial7, 'ch':fnirs_trial8, 'st':fnirs_trial9,'dt':fnirs_trial10}

In [28]:
data_dir_sync = 'Sync_txt_file/OA_302_B_WMP/P1'
sync_emg = pd.read_csv(data_dir_sync+'/EMGtime.txt', sep='\s+')
sync_fnirs = pd.read_csv(data_dir_sync+'/fNIRstime.txt', sep='\s+')
sync_tm = pd.read_csv(data_dir_sync+'/treadmill_v.txt', sep='\s+')

In [39]:
fNIRS['nw1']

,time,subject_ID,cohort,block,trial_i,HbO1,HbR1,HbR2,HbO3,HbR3,...,HbO12,HbR12,HbO13,HbR13,HbO14,HbR14,HbO15,HbR15,HbO16,HbR16
0,0.0,302,3,4,1,0.4448,-0.148700,0.4951,-0.12040,0.2088,...,0.3386,0.2854,0.3441,0.09651,0.1313,0.1924,7.276,-7.491,0.09617,0.2183
1,0.5,302,3,4,1,0.4789,-0.103200,0.5122,-0.07613,0.2308,...,0.4285,0.2520,0.3548,0.08879,0.1912,0.1843,6.254,-5.931,0.11840,0.1981
2,1.0,302,3,4,1,0.5005,-0.060990,0.5221,-0.03719,0.2475,...,0.5005,0.2205,0.3570,0.07543,0.2445,0.1748,5.144,-4.443,0.13770,0.1764
3,1.5,302,3,4,1,0.5093,-0.024520,0.5253,-0.00524,0.2579,...,0.5533,0.1912,0.3513,0.05776,0.2916,0.1637,4.049,-3.117,0.15450,0.1541
4,2.0,302,3,4,1,0.5049,0.004962,0.5213,0.01908,0.2611,...,0.5853,0.1636,0.3375,0.03753,0.3313,0.1505,3.047,-2.014,0.16790,0.1320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,362.0,302,3,4,1,-2.5490,1.666000,-3.5910,1.71900,-1.7310,...,-5.2240,1.0990,-4.1090,-0.68250,-5.4580,-0.9895,14.39,-52.82,-2.51800,0.1533
725,362.5,302,3,4,1,-2.4480,1.579000,-3.4930,1.65600,-1.6840,...,-5.0540,1.0470,-4.0040,-0.59730,-5.2920,-0.9231,17.41,-55.62,-2.34800,0.2070
726,363.0,302,3,4,1,-2.2950,1.451000,-3.3400,1.55600,-1.6070,...,-4.8910,1.0070,-3.9250,-0.47050,-5.1500,-0.8289,20.43,-58.01,-2.20000,0.3005
727,363.5,302,3,4,1,-2.1060,1.293000,-3.1500,1.42600,-1.5090,...,-4.7510,0.9830,-3.8820,-0.30600,-5.0480,-0.7139,23.45,-59.97,-2.08700,0.4313


In [38]:
sync_fnirs

,SCDtime,BStime,BEtime,RStime,RETime
0,8.013938,24.564213,34.449446,35.10843,279.202133


In [8]:
class OA_data:
    def __init__(self, number):
        self.data_array = []
        self.number = number
        
        
    def fit_emg(self, data_dir = 'csv/'):
        trial_name_1 = self.number + "_NW1"
        trial_name_2 = self.number + "_NW2"
        trial_name_3 = self.number + "_P1"
        trial_name_4 = self.number + "_P2"
        
        trial_data = glob.glob(data_dir + trial_name_1 + "*")
        try:
            trial_name = trial_data[-1]
            self.df_emg1 = load_data(trial_name, data_dir = '')
        except:
            print("EMG file for " + trial_name_1 + " does not exist")
            
        trial_data = glob.glob(data_dir + trial_name_2 + "*")
        try:
            trial_name = trial_data[-1]
            self.df_emg2 = load_data(trial_name, data_dir = '')
        except:
            print("EMG file for " + trial_name_2 + " does not exist")
            
        trial_data = glob.glob(data_dir + trial_name_3 + "*")
        try:
            trial_name = trial_data[-1]
            self.df_emg3 = load_data(trial_name, data_dir = '')
        except:
            print("EMG file for " + trial_name_3 + " does not exist")
            
        trial_data = glob.glob(data_dir + trial_name_4 + "*")
        try:
            trial_name = trial_data[-1]
            self.df_emg4 = load_data(trial_name, data_dir = '')
        except:
            print("EMG file for " + trial_name_4 + " does not exist")
    
    def fit_fNIRS(self, data_dir = 'fNIRS_Walking data'):
        file_name = '/OA_FNIRS_2019_WALK_' + self.number + '_oxydata.txt'
        self.df_fNIRS = pd.read_csv(data_dir+file_name,sep='\t')
    
    def fit_TMdata(self, data_dir = 'Treadmill_data'):
        trial_name_1 = self.number + "_N1"
        trial_name_2 = self.number + "_N2"
        trial_name_3 = self.number + "_P1"
        trial_name_4 = self.number + "_P2"
        
        file_name = '/OA_' + trial_name_1 + '_RAWDATA.csv'
        self.df_tm1 = load_data(file_name, data_dir)
        self.df_tm1 = self.df_tm1.rename(columns={'Time': 'time'})
        
        file_name = '/OA_' + trial_name_2 + '_RAWDATA.csv'
        self.df_tm2 = load_data(file_name, data_dir)
        self.df_tm2 = self.df_tm2.rename(columns={'Time': 'time'})
        
        file_name = '/OA_' + trial_name_3 + '_RAWDATA.csv'
        self.df_tm3 = load_data(file_name, data_dir)
        self.df_tm3 = self.df_tm3.rename(columns={'Time': 'time'})
        
        file_name = '/OA_' + trial_name_4 + '_RAWDATA.csv'
        self.df_tm4 = load_data(file_name, data_dir)
        self.df_tm4 = self.df_tm4.rename(columns={'Time': 'time'})
        
    def save(self):
        hdf = pd.HDFStore(self.number +'.h5')
        hdf.put('emg_N1', self.df_emg1, format='table', data_columns=True)
        hdf.put('emg_N2', self.df_emg2, format='table', data_columns=True)
        hdf.put('emg_P1', self.df_emg3, format='table', data_columns=True)
        hdf.put('emg_P2', self.df_emg4, format='table', data_columns=True)
        hdf.put('fNIRS', self.df_fNIRS, format='table', data_columns=True)
        hdf.put('TM_N1', self.df_tm1, format='table', data_columns=True)
        hdf.put('TM_N2', self.df_tm2, format='table', data_columns=True)
        hdf.put('TM_P1', self.df_tm3, format='table', data_columns=True)
        hdf.put('TM_P2', self.df_tm4, format='table', data_columns=True)
        hdf.close()

In [9]:
abc = OA_data('201')

In [10]:
abc.fit_emg()
abc.fit_fNIRS()
abc.fit_TMdata()

In [11]:
abc.save()

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning:

object name is not a valid Python identifier: 'X[s]'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning:

object name is not a valid Python identifier: 'R RECTUS FEMORIS: EMG 1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning:

object name is not a valid Python identifier: 'X[s].1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/attributeset.py:464: NaturalNameWarning:

object name is not a valid Python identifier: 'X[s].21_kind'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/attributeset.py:464: NaturalNameWarning:

object name is not a valid Python identifier: 'X[s].21_meta'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/attributeset.py:464: NaturalNameWarning:

object name is not a valid Python identifier: 'X[s].21_dtype'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/attributeset.py:464: NaturalNameWarning:

object name is not a valid Python identifier: 'X[s].45_kind'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/attributeset.py:464: NaturalNameWarning:

object name is not a valid Python identifier: 'X[s].45_meta'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/attributeset.py:464: NaturalNameWarning:

object name is not a valid Python identifier: 'X[s].45_dtype'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning:

object name is not a valid Python identifier: '1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning:

object name is not a valid Python identifier: '2'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning:

object name is not a valid Python identifier: '3'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/env

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning:

object name is not a valid Python identifier: 'sep='; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning:

object name is not a valid Python identifier: 'Unnamed: 1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/attributeset.py:464: NaturalNameWarning:

object name is not a valid Python identifier: 'sep=_kind'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though



In [12]:
hdf = pd.HDFStore('201.h5')

In [13]:
hdf.emg_N1

,X[s],R RECTUS FEMORIS: EMG 1,X[s].1,R RECTUS FEMORIS: Acc 1.X,X[s].2,R RECTUS FEMORIS: Acc 1.Y,X[s].3,R RECTUS FEMORIS: Acc 1.Z,X[s].4,L RECTUS FEMORIS: EMG 2,...,X[s].56,Trigno Trigger sensor 15: Trig 15,X[s].57,L PECTORALIS MAJOR: EKG 16,X[s].58,L PECTORALIS MAJOR: Acc 16.X,X[s].59,L PECTORALIS MAJOR: Acc 16.Y,X[s].60,L PECTORALIS MAJOR: Acc 16.Z
0,0.000000,0.0,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0,...,0.000000,0.072555,0.000000,0.0,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000
1,0.000519,0.0,0.00675,0.000000,0.00675,0.000000,0.00675,0.000000,0.000519,0.0,...,0.000519,0.072555,0.000519,0.0,0.00675,0.000000,0.00675,0.000000,0.00675,0.000000
2,0.001038,0.0,0.01350,0.000000,0.01350,0.000000,0.01350,0.000000,0.001038,0.0,...,0.001038,0.072555,0.001038,0.0,0.01350,0.000000,0.01350,0.000000,0.01350,0.000000
3,0.001558,0.0,0.02025,0.000000,0.02025,0.000000,0.02025,0.000000,0.001558,0.0,...,0.001558,0.072555,0.001558,0.0,0.02025,0.000000,0.02025,0.000000,0.02025,0.000000
4,0.002077,0.0,0.02700,-0.966812,0.02700,0.096681,0.02700,0.068482,0.002077,0.0,...,0.002077,0.072555,0.002077,0.0,0.02700,0.096681,0.02700,-0.765393,0.02700,0.036255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383833,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
383834,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
383835,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
383836,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000


In [177]:
hdf.close()

In [6]:
import glob
import os

In [142]:
glob.glob('Treadmill_data/*')

['Treadmill_data/OA_201_N1_RAWDATA.csv',
 'Treadmill_data/OA_201_N2_RAWDATA.csv',
 'Treadmill_data/OA_201_P1_RAWDATA.csv',
 'Treadmill_data/OA_201_P2_RAWDATA.csv',
 'Treadmill_data/OA_202_N1_RAWDATA.csv',
 'Treadmill_data/OA_202_N2_RAWDATA.csv',
 'Treadmill_data/OA_202_P1_RAWDATA.csv',
 'Treadmill_data/OA_202_P2_RAWDATA.csv',
 'Treadmill_data/OA_203_N1_RAWDATA.csv',
 'Treadmill_data/OA_203_N2_RAWDATA.csv',
 'Treadmill_data/OA_203_P1_RAWDATA.csv',
 'Treadmill_data/OA_203_P2_RAWDATA.csv',
 'Treadmill_data/OA_302_N1_RAWDATA.csv',
 'Treadmill_data/OA_302_N2_RAWDATA.csv',
 'Treadmill_data/OA_302_P1_RAWDATA.csv',
 'Treadmill_data/OA_302_P2_RAWDATA.csv']

In [6]:
hdf = pd.HDFStore('storage1.h5')

In [7]:
hdf.df_emg3

AttributeError: 'HDFStore' object has no attribute 'df_emg3'

In [110]:
hdf.put('d1', cdf, format='table', data_columns=True)

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning:

object name is not a valid Python identifier: 'X[s]'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning:

object name is not a valid Python identifier: 'R RECTUS FEMORIS: EMG 1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

/Users/ryu/opt/anaconda3/envs/oa/lib/python3.8/site-packages/tables/path.py:155: NaturalNameWarning:

object name is not a valid Python identifier: 'X[s].1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though

In [112]:
new = hdf['d1']

In [119]:
hdf.close()

In [118]:
hdf['d1']

,X[s],R RECTUS FEMORIS: EMG 1,X[s].1,R RECTUS FEMORIS: Acc 1.X,X[s].2,R RECTUS FEMORIS: Acc 1.Y,X[s].3,R RECTUS FEMORIS: Acc 1.Z,X[s].4,L RECTUS FEMORIS: EMG 2,...,X[s].56,Trigno Trigger sensor 15: Trig 15,X[s].57,L PECTORALIS MAJOR: EKG 16,X[s].58,L PECTORALIS MAJOR: Acc 16.X,X[s].59,L PECTORALIS MAJOR: Acc 16.Y,X[s].60,L PECTORALIS MAJOR: Acc 16.Z
0,0.000000,0.0,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.0,...,0.000000,0.072555,0.000000,0.0,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000
1,0.000519,0.0,0.00675,0.000000,0.00675,0.00000,0.00675,0.000000,0.000519,0.0,...,0.000519,0.072555,0.000519,0.0,0.00675,0.000000,0.00675,0.000000,0.00675,0.000000
2,0.001038,0.0,0.01350,0.000000,0.01350,0.00000,0.01350,0.000000,0.001038,0.0,...,0.001038,0.072555,0.001038,0.0,0.01350,0.000000,0.01350,0.000000,0.01350,0.000000
3,0.001558,0.0,0.02025,0.000000,0.02025,0.00000,0.02025,0.000000,0.001558,0.0,...,0.001558,0.072555,0.001558,0.0,0.02025,0.000000,0.02025,0.000000,0.02025,0.000000
4,0.002077,0.0,0.02700,-0.958755,0.02700,0.12488,0.02700,0.016114,0.002077,0.0,...,0.002077,0.072555,0.002077,0.0,0.02700,0.153079,0.02700,-0.789563,0.02700,0.048341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378061,NaN,0.0,NaN,0.000000,NaN,0.00000,NaN,0.000000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
378062,NaN,0.0,NaN,0.000000,NaN,0.00000,NaN,0.000000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
378063,NaN,0.0,NaN,0.000000,NaN,0.00000,NaN,0.000000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
378064,NaN,0.0,NaN,0.000000,NaN,0.00000,NaN,0.000000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000


In [82]:
col = cdf.columns

In [88]:
z = pd.DataFrame(data = g, columns = col)
z

,X[s],R RECTUS FEMORIS: EMG 1,X[s].1,R RECTUS FEMORIS: Acc 1.X,X[s].2,R RECTUS FEMORIS: Acc 1.Y,X[s].3,R RECTUS FEMORIS: Acc 1.Z,X[s].4,L RECTUS FEMORIS: EMG 2,...,X[s].56,Trigno Trigger sensor 15: Trig 15,X[s].57,L PECTORALIS MAJOR: EKG 16,X[s].58,L PECTORALIS MAJOR: Acc 16.X,X[s].59,L PECTORALIS MAJOR: Acc 16.Y,X[s].60,L PECTORALIS MAJOR: Acc 16.Z
0,0.000000,0.0,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.0,...,0.000000,0.072555,0.000000,0.0,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000
1,0.000519,0.0,0.00675,0.000000,0.00675,0.00000,0.00675,0.000000,0.000519,0.0,...,0.000519,0.072555,0.000519,0.0,0.00675,0.000000,0.00675,0.000000,0.00675,0.000000
2,0.001038,0.0,0.01350,0.000000,0.01350,0.00000,0.01350,0.000000,0.001038,0.0,...,0.001038,0.072555,0.001038,0.0,0.01350,0.000000,0.01350,0.000000,0.01350,0.000000
3,0.001558,0.0,0.02025,0.000000,0.02025,0.00000,0.02025,0.000000,0.001558,0.0,...,0.001558,0.072555,0.001558,0.0,0.02025,0.000000,0.02025,0.000000,0.02025,0.000000
4,0.002077,0.0,0.02700,-0.958755,0.02700,0.12488,0.02700,0.016114,0.002077,0.0,...,0.002077,0.072555,0.002077,0.0,0.02700,0.153079,0.02700,-0.789563,0.02700,0.048341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378061,NaN,0.0,NaN,0.000000,NaN,0.00000,NaN,0.000000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
378062,NaN,0.0,NaN,0.000000,NaN,0.00000,NaN,0.000000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
378063,NaN,0.0,NaN,0.000000,NaN,0.00000,NaN,0.000000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
378064,NaN,0.0,NaN,0.000000,NaN,0.00000,NaN,0.000000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000


In [87]:
cdf.equals(z)

False

In [46]:
os.getcwd()

'/Users/ryu/Box/Summer Data Science_2020/For Ryu'

In [52]:
a = glob.glob("csv/201_NW1*")

In [54]:
load_data('302_NW1_Rep_1.2.csv', 'csv/' )

,X[s],R RECTUS FEMORIS: EMG 1,X[s].1,R RECTUS FEMORIS: Acc 1.X,X[s].2,R RECTUS FEMORIS: Acc 1.Y,X[s].3,R RECTUS FEMORIS: Acc 1.Z,X[s].4,L RECTUS FEMORIS: EMG 2,...,X[s].56,Trigno Trigger sensor 15: Trig 15,X[s].57,L PECTORALIS MAJOR: EKG 16,X[s].58,L PECTORALIS MAJOR: Acc 16.X,X[s].59,L PECTORALIS MAJOR: Acc 16.Y,X[s].60,L PECTORALIS MAJOR: Acc 16.Z
0,0.000000,0.0,0.00000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.0,...,0.000000,0.072555,0.000000,0.0,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000
1,0.000519,0.0,0.00675,0.000000,0.00675,0.000000,0.00675,0.00000,0.000519,0.0,...,0.000519,0.072555,0.000519,0.0,0.00675,0.000000,0.00675,0.000000,0.00675,0.000000
2,0.001038,0.0,0.01350,0.000000,0.01350,0.000000,0.01350,0.00000,0.001038,0.0,...,0.001038,0.072555,0.001038,0.0,0.01350,0.000000,0.01350,0.000000,0.01350,0.000000
3,0.001558,0.0,0.02025,0.000000,0.02025,0.000000,0.02025,0.00000,0.001558,0.0,...,0.001558,0.072555,0.001558,0.0,0.02025,0.000000,0.02025,0.000000,0.02025,0.000000
4,0.002077,0.0,0.02700,-0.898329,0.02700,0.060426,0.02700,-0.14905,0.002077,0.0,...,0.002077,0.072555,0.002077,0.0,0.02700,0.060426,0.02700,-0.636484,0.02700,0.306157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578157,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.00000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
578158,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.00000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
578159,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.00000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
578160,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.00000,NaN,0.0,...,NaN,0.000000,NaN,0.0,NaN,0.000000,NaN,0.000000,NaN,0.000000
